In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [5]:
#同步爬取
import time
start = time.time()
def get_page(str):
    print('正在下载：',str)
    time.sleep(2)
    print(str,'下载完成')

str_list = ['a','b','c','d']
for i in str_list:
    get_page(i)

end = time.time()

print('总共花费时间{0}second'.format(end-start))


正在下载： a
a 下载完成
正在下载： b
b 下载完成
正在下载： c
c 下载完成
正在下载： d
d 下载完成
总共花费时间8.023643255233765second


In [4]:
#异步爬虫的方式：
#1、多线程
#2、进程池或线程池

import time
from multiprocessing.dummy import Pool

start = time.time()

def get_page(str):
    print('正在下载：',str)
    time.sleep(2)
    print(str,'下载完成')

str_list = ['a','b','c','d']
pool = Pool(4)
pool.map(get_page,str_list)
end = time.time()

print('总共花费时间{0}second'.format(end-start))

正在下载：正在下载： 正在下载：正在下载：  ac

b
 d
bda 下载完成
 下载完成
 下载完成
c 下载完成


[None, None, None, None]

总共花费时间2.014364004135132second


In [55]:
#线程池

import requests
from lxml import etree
from multiprocessing.dummy import Pool
url = 'https://www.pearvideo.com/category_5'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

text_1 = requests.get(url=url,headers=headers).text

tree = etree.HTML(text_1)
video_url = tree.xpath('//div[@class="listvideo-list-bd"]//li[@class="categoryem "]//a[@class="vervideo-lilink actplay"]/@href')
video_name = tree.xpath('//div[@class="listvideo-list-bd"]//a[@class="vervideo-lilink actplay"]//div[@class="vervideo-title"]/text()')
video_url,video_name
def get_video(url,name):
    url = 'https://www.pearvideo.com/'+url
    text_ = requests.get(url=url,headers=headers).text
    tree_ = etree.HTML(text_)
    video_url = tree_.xpath('//div[@class="img prism-player"]/')
    print(video_url)
#     content = requests.get(url=video_url,headers=headers)
#     with open('./爬虫数据/'+name+'.mp4','wb')as f:
#         f.write(content_)
        
def fun(i):
    get_video(i[0],i[1])

        
# element = [(video_url[i],video_name[i]) for i in range(len(video_url))]
# pool = Pool(4)
# pool.map(fun,element)
        
    
url = 'https://www.pearvideo.com/videoStatus.jsp?contId=1737017&mrd=0.042480902044259805'
text = requests.get(url=url,headers=headers).text
text

(['video_1737017', 'video_1737036', 'video_1737030'],
 ['泳坛男神汪顺奥运夺金！曾趣答萌娃提问：游泳想放屁咋办',
  '芦玉菲吊杠母亲心疼落泪，队医称她一直被伤病困扰',
  '李宗伟回忆林李大战，林丹是最伟大的对手',
  '中国“体操摇篮”里的孩子：哭着说不练却继续坚持'])

'\n{\n\t"resultCode":"5",\n\t"resultMsg":"该文章已经下线！",\n\t"systemTime": "1627660296802"\n}\n'

In [13]:
#单线程+异步协程
import asyncio

async def requests(url):
    print('请求的url是：',url)
    print('请求成功')
    return url
#创建协程对象
c = requests('www.baidu.com')

# #创建事件循环对象
# loop = asyncio.get_event_loop()
# #将协程对象注册到loop中，然后启动loop
# loop.run_until_complete(c)

# loop = asyncio.get_event_loop()
# #创建task对象
# task = loop.create_task(c)
# print(task)
# loop.run_until_complete(task)
# print(task)

# loop = asyncio.get_event_loop()
# #创建future
# task = asyncio.ensure_future(c)
# print(task)
# loop.run_until_complete(task)
# print(task)

#绑定回调
def  callback_func(task):
    print(task.result)
loop = asyncio.get_event_loop()
task = asyncio.ensure_future(c)
#返回的对象默认调用的是task的result，task的返回值是c中弹出的url
task.add_done_callback(callback_func)
loop.run_until_complete(task)

请求的url是： www.baidu.com
请求成功
<built-in method result of _asyncio.Task object at 0x000001C9F7D1D900>


'www.baidu.com'

In [19]:
#多任务异步协程实现 

import asyncio
import time

async def request(url):
    print('正在下载',url)
#     time.sleep(2)
    await asyncio.sleep(2)
    print('下载完毕')
    
start = time.time()
urls = [
    'www.baidu.com', 
    'www.sougou.com',
    'www.google.com'
]

#任务列表
stack = []
for url in urls:
    c = request(url)
    task = asyncio.ensure_future(c)
    stack.append(task)
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(stack))

end = time.time()
print(end-start,' seconds')


正在下载 www.baidu.com
正在下载 www.sougou.com
正在下载 www.google.com
下载完毕
下载完毕
下载完毕
2.010319471359253  seconds


In [32]:
#aiohttp 异步请求 
import requests
import aiohttp
import asyncio

urls = ['https://cn.bing.com/academic','https://cn.bing.com/dict','https://cn.bing.com/maps']
async def get_page(url):
    async with aiohttp.ClientSession() as session:
        #.get()  .post()  headers params data proxy
        async with await session.get(url) as response:
            page_text = await response.text()
            print('page_text length is ',len(page_text))
            
start = time.time()            

stack = []
for url in urls:
    c = get_page(url)
    task = asyncio.ensure_future(c)
    stack.append(task)
    
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(stack))

end = time.time()

print('总耗时',end-start)

page_text length is  110912
page_text length is  147017
page_text length is  120456
总耗时 0.8490839004516602
